<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/IntroduccionStableBaselines3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Introducción a Stable-Baselines3 - Semana 3 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/IntroduccionStableBaselines3.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>


Stable-Baselines3 (SB3) es una librería basada en PyTorch que contiene la implementación de múltiples algoritmos de Aprendizaje por Refuerzo. Stable-Baselines3, versión mejorada de Stable-Baselines, es actualmente una de las librerías mejor actualizadas y documentadas en el campo de Aprendizaje por Refuerzo debido a su naturaleza open-source. Esta librería ofrece implementaciones limpias de diferentes algoritmos como PPO, A2C, SAC, DQN, entre otros, que son fáciles de utilizar con entornos de Gymnasium.

En este notebook se presenta una traducción directa del tutorial oficial de Stable Baselines <a href="https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/1_getting_started.ipynb">Stable Baselines3 Tutorial - Getting Started</a>. Se sugiere revisar otros tutoriales oficiales que pueden ser encontrados en <a href="https://github.com/araffin/rl-tutorial-jnrr19">este repositorio de GitHub</a>

# Tutorial de Stable Baselines3 - Primeros Pasos

Repositorio de tutoriales en Github: https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3/

Repositorio oficial de Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentación oficial de Stable-Baselines3: https://stable-baselines3.readthedocs.io/en/master/

Otras contribuciones de SB3: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

Repositorio de RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) es un marco de entrenamiento para problemas Aprendizaje por Refuerzo (RL) utilizando Stable-Baselines3. Este proporciona scripts para realizar el entrenamiento, evaluación de agentes, ajuste de hiperparámetros, gráfica de resultados y grabación de videos.  

## Introducción

En este notebok, aprenderás los elementos más básicos necesarios para utilizar la librería stable_basleines: cómo crear un modelo de RL, entrenarlo y evalaurlo. Debido a que todos los algoritmos comparten la misma interfaz, veremos qué tan simple es cambiar de un algoritmo a otro.

## Instalar Depedencias y Stable-Baselines3 usando pip

La lista completa de dependencias puede ser encontrada en el archivo [README](https://github.com/DLR-RM/stable-baselines3) del repositorio oficial.


```
pip install stable-baselines3[extra]
```

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # Para visualización
!pip install "stable-baselines3[extra]>=2.0.0a4"

## Imports

Stable-Baselines3 funciona en ambientes que siguen la [interfaz y estructura de Gym](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html).
Puede encontrar una lista de ambientes disponibles [aquí](https://gymnasium.farama.org/environments/classic_control/).

No todos los algoritmos funcionan con todos los espacios de acciones definidos; puede encontrar más información en [esta tabla](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)

In [ ]:
import gymnasium as gym #SB3 trabaja con ambientes de gymnasium
import numpy as np

Lo primero que necesitamos hacer es importar el modelo de RL. Puede revisar la documentación para saber qué modelo se puede utilizar para cada problema. En este tutorial se utilizará el algoritmo de `PPO`.

In [ ]:
from stable_baselines3 import PPO #Se importa el algoritmo PPO de SB3

El siguiente paso es importar la clase de la política que será utilizada para crear las redes neuronales (para la política/funciones de valor). Este paso en realidad es opcional, ya que se pueden usar strings directamente en los contructores. Por ejemplo:

```PPO('MlpPolicy', env)``` en lugar de ```PPO(MlpPolicy, env)```

Note también que algunos algoritmos como `SAC` tienen su propia `MlpPolicy`, y es por eso que utilizar los strings para la política es la opción recomendada.

In [ ]:
from stable_baselines3.ppo.policies import MlpPolicy

## Crea el ambiente de Gym e instancia el agente

Para este ejemplo, utilizaremos el ambiente de CartPole, un problema muy popular de control que ya fue visto anteriormente.

"Un poste es unido mediante una junta sin actuadores a un carrito, el cual se mueve por una superficie sin fricción. El sistema se conrola aplicando una fuerza de +1 o -1 al carrito. El péndulo comienza en posición erquida hacia arriba, y el objetivo es prevenir que caiga. Se recibe una recompensa de +1 por cada paso de tiempo que el poste permanece hacia arriba."

Cartpole environment: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)


Para este caso escogemos la MlpPolicy porque el espacio de observación del CartPole es un vector de características, no imágenes.

El tipo de acción que se usa (discreta/continua) será deducida de forma automática a partir del espacio de acciones del ambiente.

Aquí usaremos el algoritmo de [Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html), el cual es un método Actor-Crítico: usa una función de valor para mejorar el descento de gradiente de la política (recudicendo la varianza). Este algoritmo combina las ideas de [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html) (disponiendo de múltiples trabajadores que usan bonos de entropía para formentar la exploración) y [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) (usa una región de confianza para mejorar la estabilidad y evitar caídas catastróficas de rendimiento).

PPO es un algoritmo on-policy, lo cual significa que las trayectorias utilizadas para acutalizar las redes neuronales deben ser recolectadas utilizando la última política actualizada. Es en realidad menos eficiente en términos de muestras que los algoritmos off-policy como [DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html) o [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html), pero es mucho más rápido en términos reales de tiempo.

In [ ]:
env = gym.make("CartPole-v1") #Se crea el ambiente

model = PPO(MlpPolicy, env, verbose=0) #Se instancia el algoritmo
#model = PPO('MlpPolicy', env, verbose=0) #Alternativa recomendada

Creamos una función adicional para evaluar el agente:

In [ ]:
from stable_baselines3.common.base_class import BaseAlgorithm

def evaluate(
    model: BaseAlgorithm,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
    """
    Evalua un agente de RL por 'num_episodes' episodios.

    Lista de parámetros
    :model: el agente de RL
    :env: el ambiente de Gym
    :num_episodes: número de episodios para evaluar el agente (100 por defecto)
    :deterministic: si utilizar acciones determinísticas o estocásticas (True por defecto

    :return: recompensa promedio en los `num_episodes` episodios (float)
    """
    # Esta función funciona para un único ambiente
    vec_env = model.get_env()
    obs = vec_env.reset() #reinicia el ambiente
    all_episode_rewards = [] #lista vacía para almacenar recompensas
    for _ in range(num_episodes): #ciclo de num_episodes episodios
        episode_rewards = [] #almacena las recompensas del episodio
        done = False #inicializa condición de terminación

        # Nota: SB3 VecEnv se reinicia automáticamente:
        # https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api
        # obs = vec_env.reset()

        while not done: #ejecuta un episodio
            # _states son sólo útiles cuando se usan políticas LSTM
            # `deterministic` es para usar acciones determinísticas
            action, _states = model.predict(obs, deterministic=deterministic) #el modelo predice qué acción elegir en base al pultimo estado observado
            # en este caso action, rewards y dones son arreglos
            # porque se está usando un ambiente vectorizado
            obs, reward, done, _info = vec_env.step(action) #da el paso y observa el nuevo estado y recompensa
            episode_rewards.append(reward) #almacena recompensa

        all_episode_rewards.append(sum(episode_rewards)) #calcula el retorno del episodio (suma) y lo almacena en la lista

    mean_episode_reward = np.mean(all_episode_rewards) #calcula la recompensa promedio
    print(f"Mean reward: {mean_episode_reward:.2f} - Num episodes: {num_episodes}") #da un print con la recompensa promedio

    return mean_episode_reward #retorna la recompensa promedio

Evaluemos un agente sin entrenar, este debería ser un agente aleatorio.

In [ ]:
# Agente aleatorio, antes del entrenamiento
mean_reward_before_train = evaluate(model, num_episodes=100, deterministic=True)

No obstante, Stable-Baselines3 ya tiene incluida una función similar para evaluar agentes:

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy #importa la función evaluate_policy para evaluar agentes directamente

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False) #obtiene la recompensa promedio y desviación estándar

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}") #imprime resultados

## Entrenar y evaluar el agente

In [ ]:
# Entrena el agent por 10000 pasos
model.learn(total_timesteps=10_000) #usa model.learn para entrenar el agente

In [ ]:
# Evalúa el agente entrenado
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100) #obtiene la recompensa promedio y desviación estándar

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}") #imprime resultados

!Aparentemente el entrenamiento funcionó porque la recompensa incrementó mucho¡

### Preparar grabación de video

En este tutorial se muestra la forma de grabar un video utilizando el wrapper [VecVideoRecorder](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper. Se realiza una explicación sobre qué son los wrappers y cómo funcionan en [este segundo tutorial](https://github.com/araffin/rl-tutorial-jnrr19/blob/sb3/2_gym_wrappers_saving_loading.ipynb).

In [ ]:
# Hay que configurar fake display; de lo contrario el redner fallará
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Tomado de https://github.com/eleurent/highway-env

    Lista de parámetros
    :video_path: (str)Ruta a la carpeta que contiene los videos
    :prefix: (str) Filtra el video, mostrando sólo los que inician con este prefijo
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Comienza el video en step=0 y graba 500 pasos
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Cierra la grabadora de video
    eval_env.close()

# Visualiza el agente entrenado

In [ ]:
record_video("CartPole-v1", model, video_length=500, prefix="ppo-cartpole") #graba el video

In [ ]:
show_videos("videos", prefix="ppo") #muestra el video

## EXTRA: Entrena un modelo de RL en una sóla línea

La clase de política a usar será inferida y el ambiente será automáticamente creado. Esto funciona porque ambos se encuentran [registrados](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html).

In [ ]:
# Entrena un modelo en una sola línea de código
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

# Conclusión

In este notebook hemos aprendido:
- Cómo definir u entrenar un modelo de RL utilizando stable_baselines3, sólo se necesita una línea de código ;)